## BERT Knowledge Representation
We want to compute, at each step of the user session (i.e. each document clicked), how their internal knowledge representation changes. Therefore, we have a few different methods to do so using Bert embeddings as a starting point.
A few assumptions:
- user starts with an empty knowledge representation
- User READS every document, and that is added to their knowledge

Document embeddings for BERT can come in a few different forms. Check `Compute_BERT_embeddings.ipynb` for how we compute each:
- SUM: sum of the embeddings for each sentence of the document
- MEAN: mean of the embeddings for each sentence of the document
- TRUNC: Truncate the document at the first 384 tokens.
- maxp_pairwise: Considering all sentences from the Wikipedia topic and the document, consider only the sentence with the higher similarity for any Wikipedia sentence
- maxp_sum: Consider only the sentence with higher similarity to the SUM of the wikipedia sentences
- maxp_mean: Consider only the sentence with higher similarity to the MEAN of the wikipedia sentences
- maxp_trunc: Consider only the sentence with higher similarity to the truncated wikipedia document
    
These are the ways we can compute the users' knowledge evolution. Will be compared to the same method of aggregation on the Wikipedia text

- MEAN: Concatenate all of the documents, the MEAN of these is the final knowledge.
- SUM: As the user clicks on documents, SUM the embeddings


In [32]:
import pickle
import urllib.parse
from collections import defaultdict
import json
import numpy as np
from tqdm.auto import tqdm


def normalize_vector(v):
    return v / np.linalg.norm(v)


dataset = json.load(open("../data/logs_with_position.json"))


embeddings = {
    "docs_mean": pickle.load(open("../data/docs_mean_embeddings.pkl", "rb")),
    "docs_sum": pickle.load(open("../data/docs_sum_embeddings.pkl", "rb")),
    "docs_trunc": pickle.load(open("../data/docs_trunc_embeddings.pkl", "rb")),
    "maxp_pairwise": pickle.load(open("../data/docs_maxp_pairwise_embeddings.pkl", "rb")),
    "maxp_sum": pickle.load(open("../data/docs_maxp_sum_embeddings.pkl", "rb")),
    "maxp_mean": pickle.load(open("../data/docs_maxp_mean_embeddings.pkl", "rb")),
    "maxp_trunc": pickle.load(open("../data/docs_maxp_trunc_embeddings.pkl", "rb")),
}

FileNotFoundError: [Errno 2] No such file or directory: '../docs_maxp_trunc_embeddings.pkl'

In [ ]:
a = np.random.randn(12)
rolling_avg = 0.0


for n, i in enumerate(a):
    rolling_avg += (i - rolling_avg) / (n + 1)

True

In [31]:
methods = list(embeddings.keys())
users_knowledge_MEAN = []  # add final score for the user
users_knowledge_SUM = []

missing_docs = set()

for u in dataset:
    user_knowledge_mean = {k: np.zeros(768) for k in methods}
    user_knowledge_sum = {k: np.zeros(768) for k in methods}
    clicks = 0
    for d in u["clicks"]:
        url = d["url"]
        if url not in docs_mean or not np.any(embeddings["docs_mean"][url]):
            missing_docs.add(url)
            continue
        clicks += 1
        for method in methods:
            emb = embeddings[method][url]
            user_knowledge_mean[method] += (emb - user_knowledge_mean[method]) / (clicks)
            user_knowledge_sum[method] += emb  # normalize at the end
    # normalize and compute final knowledge gain
    for method in methods:
        user_knowledge_mean[method] = normalize_vector(user_knowledge_mean[method])
        user_knowledge_sum[method] = normalize_vector(user_knowledge_sum[method])
        
        
        
    # final scores
    for method 
    
    # print(u)
    break